In [1]:
suppressPackageStartupMessages({
    library(data.table)
    library(dplyr)
    library(glue)
    library(ggplot2)
    library(sf)
    library(patchwork)
    library(ggthemes)
    library(purrr)
    library(Matrix)
    library(tidyr)
    library(Seurat)
    library(uwot)
    library(igraph)
    library(future)
    library(scran)
    library(DESeq2)
    library(pheatmap)
    library(furrr)
    library(spatula)
    library(deldir)
    library(tidyverse)
    library(purrr)
    library(tictoc)

})
fig.size <- function(h, w) {
    options(repr.plot.height = h, repr.plot.width = w)
}


In [2]:
dmt1 <- readRDS('../data/dmt1.rds')
aggs1 <- readRDS('../data/aggs1.rds')
gene_ls = read.table('../data/VizgenLungHacohen/cells/genes.txt', header=FALSE)$V1


In [3]:
programs = read.csv('../data/NMF41.csv', header=1)
# Don't include MP24.Cilia and MP11.Translation.initiation due to non-overlapping genes
lung_programs = c('MP1..Cell.Cycle...G2.M','MP5.Stress','MP6.Hypoxia','MP2..Cell.Cycle...G1.S','MP12.EMT.I','MP18.Interferon.MHC.II..II.','MP13.EMT.II','MP14.EMT.III',
                 'MP10.Protein.maturation','MP8.Proteasomal.degradation','MP19.Epithelial.Senescence','MP3..Cell.Cylce.HMG.rich','MP23.Secreted.II','MP30.PDAC.classical')
lung_programs = programs[lung_programs]
# Get just the epithelial cells
epi_idx = which(dmt1$pts$type_lvl1=='Epithelial')
intersected_genes = sapply(lung_programs, intersect, gene_ls)

In [4]:
epi_seurat = readRDS('../data/epi_seurat.RDS')
program_score = epi_seurat@meta.data[,14:28]
weighted_adjmtx = uwot::umap(program_score, min_dist=0.3, spread=1, ret_extra='fgraph', fast_sgd=TRUE)

Found more than one class "dist" in cache; using the first, from namespace 'spam'

Also defined by ‘BiocGenerics’

Found more than one class "dist" in cache; using the first, from namespace 'spam'

Also defined by ‘BiocGenerics’



In [ ]:
source('../src/cluster_utils.R')
leiden_clusters = do_leiden_one(weighted_adjmtx$fgraph, resolution=0.4, n_starts=1)

In [ ]:
# Merge new epi subtypes with the original data
epi_idx = which(dmt1$pts$type_lvl1=='Epithelial')
epi_counts = dmt1$counts[,epi_idx]
epi_meta = dmt1$pts[epi_idx,]
epi_meta$type_lvl2 <- paste('Epi', leiden_clusters, sep='')
epi_subtypes = epi_meta[,c('ORIG_ID', 'type_lvl2')]
merge_df = merge(dmt1$pts, epi_subtypes, by='ORIG_ID', all=TRUE)
# Replace column type_lvl2 where available
merge_df$type_lvl2.x = ifelse(!is.na(merge_df$type_lvl2.y), merge_df$type_lvl2.y, merge_df$type_lvl2.x)
merge_df = select(merge_df, -type_lvl2.y)
colnames(merge_df) <- c('ORIG_ID','X','Y','type_lvl1','type_lvl2','hubID','f','agg_id','spatial_cluster')
head(merge_df)


In [16]:
#write.csv(merge_df, '../data/merge_subtype_epi.csv')

In [2]:
merge_df = read.csv('../data/merge_subtype_epi.csv')
merge_df = merge_df %>% dplyr::filter(type_lvl1 != '')

In [3]:
# Colocalization code
coloc_one_type = function(index_type, adj, y, nperm = 100, max_dist=30, compartments=NULL, verbose=TRUE) {
    if (verbose) message(index_type)
    types = unique(y)
    i_index = which(y == index_type)
    i_shuffle = setdiff(seq_len(length(y)), i_index)

    X = adj[i_index, ] %*% Matrix::sparse.model.matrix(~0+y) %>% as.matrix()
    colnames(X) = gsub('^y', '', colnames(X))
    freq = (colSums(X) / nrow(X))[types]

    freq_perm = map(seq_len(nperm), function(i) {
        set.seed(i)
        yperm = y
        if (is.null(compartments)) {
            yperm[i_shuffle] = sample(y[i_shuffle])
        } else {
            ## shuffle inside compartments, to preserve total composition within compartment
            .x = split(i_shuffle, compartments[i_shuffle]) %>%
                map(function(.i) {
                    ## CAUTION: if .i is a single number, sample will interpret it as 1:.i
                    if (length(.i) == 1) {
                        res = .i
                    } else {
                        res = sample(.i) ## shuffle non-index cells inside hub            
                    }
                   
                    names(res) = .i
                    return(res)
                }) %>%
                purrr::reduce(c)
            yperm[as.integer(names(.x))] <- y[.x]
        }

        X = adj[i_index, ] %*% Matrix::sparse.model.matrix(~0+yperm) %>% as.matrix() #%>% prop.table(1)
        colnames(X) = gsub('^yperm', '', colnames(X))
        (colSums(X) / nrow(X))[types]    
    }) %>%
        purrr::reduce(rbind2)
    
    
    # Return all 0 here
    #print(apply(freq_perm, 2, sd))

   
    stats = tibble(
        type = types,
        freq,
        zscore = (freq - apply(freq_perm, 2, mean)) / apply(freq_perm, 2, sd),
        pval = exp(pnorm(-zscore, log.p = TRUE, lower.tail = TRUE)), ## one-tailed
        fdr = p.adjust(pval)
    ) %>%
        cbind(dplyr::rename(data.frame(t(apply(freq_perm, 2, quantile, c(.025, .975)))), q025 = `X2.5.`, q975 = `X97.5.`)) %>% ## 95% CI

    
        subset(type != index_type) %>%
        dplyr::mutate(index_type = index_type) %>%
        dplyr::select(index_type, type, everything()) %>%
        arrange(fdr)

    return(stats)    
}


coloc_all_types = function(index_types, coords, y, nperm = 100, nsteps=1, max_dist=30, compartments=NULL, parallel=TRUE, verbose=TRUE) {
    if (parallel & length(index_types) > 1) {
        plan(multicore)
    } else {
        plan(sequential)
    }

    ## Define neighbors
    ## NOTE: max_dist only refers to directly adjacent neighbors
    adj = spatula::getSpatialNeighbors(coords, return_weights = TRUE)
    adj@x[adj@x > max_dist] = 0
    adj = Matrix::drop0(adj)
    adj@x = rep(1, length(adj@x))
   
    ## If nsteps>1, consider not only your adjacent neighbors
    ##   but also your neighbor's neighbors etc.
    if (nsteps > 1) {
        adj = adj + Matrix::Diagonal(n = nrow(adj)) ## add self
        for (iter in seq_len(nsteps - 1)) {
            adj = adj %*% adj
        }
        ## Ignore weights. Only care if cell is a neighbor or not
        adj@x = rep(1, length(adj@x))
       
        ## Remove self as neighbor
        adj = adj - Matrix::Diagonal(n = nrow(adj))
        adj = Matrix::drop0(adj)
    }
       
    all_stats = index_types %>%
        future_map(coloc_one_type, adj, y, nperm, max_dist, compartments, verbose, .options = furrr::furrr_options(seed = 1)) %>%
        rbindlist()  

    return(all_stats)
    
}



In [4]:
plan(multicore, workers = parallel::detectCores() - 1)


In [5]:
test = subset(merge_df, hubID==c('H260', 'H455'))
print(nrow(test))
table(merge_df$hubID)

Warning message in hubID == c("H260", "H455"):
“longer object length is not a multiple of shorter object length”


[1] 135925



  H260   H261   H265   H266   H268   H269   H270   H271   H275   H276   H280 
235646     56    116   2159    223     28    237     34   1339     28     46 
  H282   H284   H285   H286   H287   H289   H290   H292   H297   H298   H300 
    11  12778     42  70426    150    247    314     72     45    745     83 
  H302   H304   H305   H307   H308   H310   H311   H312   H315   H316   H318 
   103   5749   1540    204     88    330    302    448     57    173   2458 
  H320   H321   H322   H323   H326   H327   H331   H332   H335   H339   H340 
   148     47   1794     90    121      7     96      2     69     61   2475 
  H341   H342   H343   H345   H346   H347   H348   H349   H350   H351   H353 
    62     88   9972    403   3110    288    112   5267    201     36    110 
  H354   H358   H359   H360   H361   H362   H363   H365   H366   H367   H368 
  6553    159    528    297   1289   4891     78   2038     95     46    433 
  H369   H370   H371   H373   H374   H375   H376   H380   H382 

In [6]:
tic()
coloc_res = coloc_all_types(
    index_type = unique(test$type_lvl2),
    coords = as.matrix(test[, c('X', 'Y')]),
    y = test$type_lvl2,
    compartments = test$hubID,
    max_dist = 100
) 
toc()

Epi1

Epi3

Epi2

Epi4

MARCO+ Macrophage

Epi5

Vascular

SPP1+ Macrophage

CD8 T

Fibroblast

FOLR2+CD14+ Macrophage

NCAM1+ S100B+ SEPP1+ Myeloid

Treg

MERTK+ Macrophage

CD1C+ITGAX+ DC

FCN1+LYZ+ Macrophage

CD4 T

CXCL13+ CD4 T

MMP1+SOX4+ Myeloid

CXCL10+ Macrophage

LAMP3+CD1C+ DC

TCF7+ CD8 T

ILC

LAMP3+CCL19+ mreg DC

Plasma

FLT3+ DC

B

Mast

NK

CXCL13+ CD8 T

PLA2G7+ CCL18+ Macrophage

Epi6



1237.318 sec elapsed


In [112]:
nrow(merge_df)

[1] 607885

In [ ]:
tic()
coloc_res = coloc_all_types(
    index_type = unique(merge_df$type_lvl2),
    coords = as.matrix(merge_df[, c('X', 'Y')]),
    y = merge_df$type_lvl2,
    compartments = merge_df$hubID,
    max_dist = 100
) 
toc()


  H260   H261   H265   H266   H268   H269   H270   H271   H275   H276   H280 
235646     56    116   2159    223     28    237     34   1339     28     46 
  H282   H284   H285   H286   H287   H289   H290   H292   H297   H298   H300 
    11  12778     42  70426    150    247    314     72     45    745     83 
  H302   H304   H305   H307   H308   H310   H311   H312   H315   H316   H318 
   103   5749   1540    204     88    330    302    448     57    173   2458 
  H320   H321   H322   H323   H326   H327   H331   H332   H335   H339   H340 
   148     47   1794     90    121      7     96      2     69     61   2475 
  H341   H342   H343   H345   H346   H347   H348   H349   H350   H351   H353 
    62     88   9972    403   3110    288    112   5267    201     36    110 
  H354   H358   H359   H360   H361   H362   H363   H365   H366   H367   H368 
  6553    159    528    297   1289   4891     78   2038     95     46    433 
  H369   H370   H371   H373   H374   H375   H376   H380   H382 


  H260   H261   H265   H266   H268   H269   H270   H271   H275   H276   H280 
235646     56    116   2159    223     28    237     34   1339     28     46 
  H282   H284   H285   H286   H287   H289   H290   H292   H297   H298   H300 
    11  12778     42  70426    150    247    314     72     45    745     83 
  H302   H304   H305   H307   H308   H310   H311   H312   H315   H316   H318 
   103   5749   1540    204     88    330    302    448     57    173   2458 
  H320   H321   H322   H323   H326   H327   H331   H332   H335   H339   H340 
   148     47   1794     90    121      7     96      2     69     61   2475 
  H341   H342   H343   H345   H346   H347   H348   H349   H350   H351   H353 
    62     88   9972    403   3110    288    112   5267    201     36    110 
  H354   H358   H359   H360   H361   H362   H363   H365   H366   H367   H368 
  6553    159    528    297   1289   4891     78   2038     95     46    433 
  H369   H370   H371   H373   H374   H375   H376   H380   H382 

[1] 36256